In [ ]:
import ragas.prompt.utils as dspy_utils
import dspy
import importlib
importlib.reload(dspy_utils)

##### TODO: bind ragas llm and dspy llm together

In [15]:
lm = dspy.LM('openai/gpt-4o-mini')
dspy.configure(lm=lm)

## optimize LLMContextPrecisionWithReference

In [32]:
from ragas.llms import LangchainLLMWrapper
from langchain_openai import ChatOpenAI
llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o-mini"))

In [33]:
from ragas.metrics import LLMContextPrecisionWithReference
context_precision = LLMContextPrecisionWithReference(llm=llm)
prompts = context_precision.get_prompts()["context_precision_prompt"]

In [34]:
prompts.instruction
prompts.input_model
prompts.output_model
prompts.examples

[(QAC(question='What can you tell me about Albert Einstein?', context="Albert Einstein (14 March 1879 – 18 April 1955) was a German-born theoretical physicist, widely held to be one of the greatest and most influential scientists of all time. Best known for developing the theory of relativity, he also made important contributions to quantum mechanics, and was thus a central figure in the revolutionary reshaping of the scientific understanding of nature that modern physics accomplished in the first decades of the twentieth century. His mass–energy equivalence formula E = mc2, which arises from relativity theory, has been called 'the world's most famous equation'. He received the 1921 Nobel Prize in Physics 'for his services to theoretical physics, and especially for his discovery of the law of the photoelectric effect', a pivotal step in the development of quantum theory. His work is also known for its influence on the philosophy of science. In a 1999 poll of 130 leading physicists worl

In [35]:
sig=dspy_utils.base_model_to_signature(input=prompts.input_model, output=prompts.output_model)

In [36]:
data= dspy_utils.ragas_example_to_dspy(data=prompts.examples, input_model=prompts.input_model, output_model=prompts.output_model)

In [37]:
context_precision_judge =dspy.Predict(sig)

In [38]:
from dspy.teleprompt import BootstrapFewShot
fewshot_optimizer = BootstrapFewShot(metric=dspy_utils.judge, max_bootstrapped_demos=10, max_labeled_demos=2, max_rounds=2)
fewshot_optimizer_compiled = fewshot_optimizer.compile(context_precision_judge, trainset=data)

100%|██████████| 3/3 [00:00<00:00, 331.64it/s]

Bootstrapped 2 full traces after 3 examples in round 1.


In [39]:
from dspy.teleprompt import COPRO
copro_optimizer = COPRO(
    metric=dspy_utils.judge,
    verbose=True,
)
kwargs = dict(num_threads=5, display_progress=False, display_table=0)
copro_optimizer_compiled = copro_optimizer.compile(context_precision_judge, trainset=data, eval_kwargs=kwargs)

In [40]:
examples = dspy_utils.dspy_example_to_ragas(fewshot_optimizer_compiled.demos, input_model=prompts.input_model, output_model=prompts.output_model)

In [41]:
prompts.instruction= copro_optimizer_compiled.signature.instructions
prompts.examples = examples

In [42]:
context_precision.set_prompts(**{"context_precision_prompt": prompts})

In [ ]:
context_precision.save_prompts(path="./")

In [27]:
from datasets import load_dataset
from ragas import EvaluationDataset
dataset = load_dataset("explodinggradients/amnesty_qa","english_v3")
eval_dataset = EvaluationDataset.from_hf_dataset(dataset["eval"])

Repo card metadata block was not found. Setting CardData to empty.


In [43]:
await context_precision.single_turn_ascore(eval_dataset[0])

0.9999999999666667

In [ ]:
sum([x['cost'] for x in lm.history if x['cost'] is not None])